## Context Relevance Evaluations

In many ways, feedbacks can be thought of as LLM apps themselves. Given text, they return some result. Thinking in this way, we can use TruLens to evaluate and track our feedback quality. We can even do this for different models (e.g. gpt-3.5 and gpt-4) or prompting schemes (such as chain-of-thought reasoning).

This notebook follows an evaluation of a set of test cases. You are encouraged to run this on your own and even expand the test cases to evaluate performance on test cases applicable to your scenario or domain.

In [ ]:
# Import relevance feedback function
from trulens_eval.feedback import GroundTruthAgreement, OpenAI, LiteLLM
from trulens_eval import TruBasicApp, Feedback, Tru, Select
from test_cases import context_relevance_golden_set

import openai

Tru().reset_database()

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["COHERE_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."
os.environ["ANTHROPIC_API_KEY"] = "..."
os.environ["TOGETHERAI_API_KEY"] = "..."

In [3]:
# GPT 3.5
turbo = OpenAI(model_engine="gpt-3.5-turbo")

def wrapped_relevance_turbo(input, output):
    return turbo.qs_relevance(input, output)
def wrapped_relevance_with_cot_turbo(input, output):
    return turbo.qs_relevance_with_cot_reasons(input, output)

# GPT 4
gpt4 = OpenAI(model_engine="gpt-4")

def wrapped_relevance_gpt4(input, output):
    return gpt4.qs_relevance(input, output)
def wrapped_relevance_with_cot_gpt4(input, output):
    return gpt4.qs_relevance_with_cot_reasons(input, output)

# Cohere
command_nightly = LiteLLM(model_engine="command-nightly")
def wrapped_relevance_command_nightly(input, output):
    return command_nightly.qs_relevance(input, output)

# Anthropic
claude_1 = LiteLLM(model_engine="claude-instant-1")
def wrapped_relevance_claude1(input, output):
    return claude_1.qs_relevance(input, output)

claude_2 = LiteLLM(model_engine="claude-2")
def wrapped_relevance_claude2(input, output):
    return claude_2.qs_relevance(input, output)

# Meta
llama_2_13b = LiteLLM(model_engine="togethercomputer/llama-2-7b")
def wrapped_relevance_llama2(input, output):
    return llama_2_13b.qs_relevance(input, output)

Here we'll set up our golden set as a set of prompts, responses and expected scores stored in `test_cases.py`. Then, our numeric_difference method will look up the expected score for each prompt/response pair by **exact match**. After looking up the expected score, we will then take the L1 difference between the actual score and expected score.

In [4]:
# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(context_relevance_golden_set)
# Call the numeric_difference method with app and record
f_groundtruth = Feedback(ground_truth.numeric_difference, name = "Context Relevance Smoke Test").on(Select.Record.calls[0].args.args[0]).on(Select.Record.calls[0].args.args[1]).on_output()

✅ In Context Relevance Smoke Test, input prompt will be set to *.__record__.calls[0].args.args[0] .
✅ In Context Relevance Smoke Test, input response will be set to *.__record__.calls[0].args.args[1] .
✅ In Context Relevance Smoke Test, input score will be set to *.__record__.main_output or `Select.RecordOutput` .


In [5]:
tru_wrapped_relevance_turbo = TruBasicApp(wrapped_relevance_turbo, app_id = "context relevance gpt-3.5-turbo", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_turbo = TruBasicApp(wrapped_relevance_with_cot_turbo, app_id = "context relevance with cot reasoning gpt-3.5-turbo", feedbacks=[f_groundtruth])

tru_wrapped_relevance_gpt4 = TruBasicApp(wrapped_relevance_gpt4, app_id = "context relevance gpt-4", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_gpt4 = TruBasicApp(wrapped_relevance_with_cot_gpt4, app_id = "context relevance with cot reasoning gpt-4", feedbacks=[f_groundtruth])

tru_wrapped_relevance_commandnightly = TruBasicApp(wrapped_relevance_command_nightly, app_id = "context relevance Command-Nightly", feedbacks=[f_groundtruth])

tru_wrapped_relevance_claude1 = TruBasicApp(wrapped_relevance_claude1, app_id = "context relevance Claude 1", feedbacks=[f_groundtruth])

tru_wrapped_relevance_claude2 = TruBasicApp(wrapped_relevance_claude2, app_id = "context relevance Claude 2", feedbacks=[f_groundtruth])

tru_wrapped_relevance_llama2 = TruBasicApp(wrapped_relevance_llama2, app_id = "context relevance Llama-2-13b", feedbacks=[f_groundtruth])

In [ ]:
for i in range(len(context_relevance_golden_set)):
    prompt = context_relevance_golden_set[i]["query"]
    response = context_relevance_golden_set[i]["response"]
    tru_wrapped_relevance_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_gpt4.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_gpt4.call_with_record(prompt, response)
    tru_wrapped_relevance_commandnightly.call_with_record(prompt, response)
    tru_wrapped_relevance_claude1.call_with_record(prompt, response)
    tru_wrapped_relevance_claude2.call_with_record(prompt, response)
    tru_wrapped_relevance_llama2.call_with_record(prompt, response)

In [7]:
Tru().get_leaderboard(app_ids=[])

,Context Relevance Smoke Test,latency,total_cost
app_id,,,
context relevance gpt-3.5-turbo,0.800000,0.066667,0.000762
context relevance gpt-4,0.786667,0.066667,0.015268
context relevance with cot reasoning gpt-4,0.786667,0.066667,0.019420
context relevance Claude 1,0.773333,0.066667,0.000000
context relevance with cot reasoning gpt-3.5-turbo,0.726667,0.066667,0.000914
context relevance Command-Nightly,0.673333,0.066667,0.000000
context relevance Claude 2,0.566667,0.066667,0.000000
context relevance Llama-2-13b,0.506667,0.066667,0.000000
